# Golf Model

In [84]:
import requests
import csv
from datetime import datetime
import pandas as pd

In [85]:
from config import api_key

api_key = api_key

In [86]:
ids_url = f'https://feeds.datagolf.com/historical-raw-data/event-list?file_format=json&key={api_key}'

In [91]:
response = requests.get(ids_url)
if response.status_code == 200:
    # Parse the JSON response
    event_list = response.json()
    # Assuming the JSON structure is a list of events
    for event in event_list:
        print(f"Event Name: {event['event_name']} - Event ID: {event['event_id']}")
else:
    print(f"Error fetching event list: {response.status_code}")

Event Name: Valero Texas Open - Event ID: 41
Event Name: Miami - Event ID: 28
Event Name: Club Car Championship at The Landings Golf & Athletic Club - Event ID: 128
Event Name: Singha Championship - Event ID: 10333
Event Name: Limpopo Championship - Event ID: 10332
Event Name: Nissan presents Chandigarh Open 2024 hosted by Chandigarh Golf Club - Event ID: 10331
Event Name: Villa Rica Classic - Event ID: 10334
Event Name: Novil Cup - Event ID: 10330
Event Name: Texas Children's Houston Open - Event ID: 20
Event Name: Totalplay Championship at Atlas Country Club - Event ID: 17
Event Name: The Galleri Classic - Event ID: 56
Event Name: Hero Indian Open - Event ID: 2024114
Event Name: Token Homemate Cup - Event ID: 10327
Event Name: Porsche Singapore Classic - Event ID: 2024113
Event Name: Kolkata Challenge - Event ID: 2024308
Event Name: Bupa Championship at Tulum - Event ID: 1
Event Name: Valspar Championship - Event ID: 475
Event Name: Hoag Classic - Event ID: 573
Event Name: Stella Art

In [92]:
events_df = pd.DataFrame(event_list)

In [93]:
events_df

,calendar_year,date,event_id,event_name,sg_categories,tour,traditional_stats
0,2024,2024-04-07,41,Valero Texas Open,yes,pga,yes
1,2024,2024-04-07,28,Miami,no,liv,basic
2,2024,2024-04-07,128,Club Car Championship at The Landings Golf & A...,no,kft,no
3,2024,2024-04-07,10333,Singha Championship,no,atgt,no
4,2024,2024-04-07,10332,Limpopo Championship,no,afr,no
...,...,...,...,...,...,...,...
1894,2017,2017-01-22,2017004,Abu Dhabi HSBC Championship,no,euro,no
1895,2017,2017-01-15,6,Sony Open in Hawaii,yes,pga,yes
1896,2017,2017-01-15,2017002,BMW SA Open hosted by City of Ekurhuleni,no,euro,no
1897,2017,2017-01-11,124,The Bahamas Great Exuma Classic at Sandals Eme...,no,kft,no


In [94]:
majors = ['The Masters', 'U.S. Open', 'The Open Championship', 'PGA Championship']

# Filter the DataFrame for the major tournaments
majors_df = events_df[events_df['event_name'].isin(majors)]

# Reset index after filtering
majors_df = majors_df.reset_index(drop=True)

# Display the DataFrame
majors_df.head()

# If you need the event_id as a dictionary:
major_event_ids = dict(zip(majors_df['event_name'], majors_df['event_id']))
print(major_event_ids)

{'PGA Championship': 33, 'The Open Championship': 100, 'U.S. Open': 26, 'The Masters': 14}


In [95]:
majors_df.head()

,calendar_year,date,event_id,event_name,sg_categories,tour,traditional_stats
0,2023,2023-09-23,10153,PGA Championship,no,afr,no
1,2023,2023-07-23,100,The Open Championship,yes,pga,yes
2,2023,2023-06-18,26,U.S. Open,yes,pga,yes
3,2023,2023-05-21,33,PGA Championship,yes,pga,yes
4,2022,2022-11-06,9788,PGA Championship,no,afr,no


In [96]:
base_url = url = f'https://feeds.datagolf.com/historical-raw-data/rounds?tour=pga&event_id=535&year=2021&file_format=json&key={api_key}'

def get_golf_stats(tour, event_id, year, api_key):
    params = {
        'tour': tour,
        'event_id': event_id,
        'year': year,
        'file_format': 'json',
        'key': api_key
    }
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Define years of interest
years_of_interest = range(2019, datetime.now().year + 1)

# Specify the event_id for the U.S. Open
event_ids = {'U.S. Open': '535'} 

In [97]:
filename = 'full_golf_data.csv'

# Open the file and write the CSV data
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Year', 'Event ID', 'Event Name', 'Player Name', 'Round', 'Score', 'SG Total', 'Driving Distance', 'GIR', 'Scrambling'])
    
    for year in years_of_interest:
        for event_name, event_id in event_ids.items():
            # Get the stats for event
            stats = get_golf_stats('pga', event_id, year, api_key)
            
            # Check if stats is not None
            if stats:
                # Process and write data to CSV
                for player in stats['scores']:
                    for round_number, round_data in player.items():
                        if round_number.startswith('round_'):
                            writer.writerow([
                                year,
                                event_id,
                                event_name,
                                player['player_name'],
                                round_number[-1],
                                round_data['score'],
                                round_data['sg_total'],
                                round_data['driving_dist'],
                                round_data['gir'],
                                round_data['scrambling']
                            ])

print("Data collection complete. CSV file created.")

Data collection complete. CSV file created.


In [98]:
# Specify the path to the CSV file
golf_data_df = pd.read_csv(filename)

# Now you can display the first few rows of the DataFrame to check it
df = golf_data_df
df

,Year,Event ID,Event Name,Player Name,Round,Score,SG Total,Driving Distance,GIR,Scrambling
0,2019,535,U.S. Open,"Rahm, Jon",1,69,4.718,311.5,0.556,0.778
1,2019,535,U.S. Open,"Rahm, Jon",2,70,3.787,315.7,0.611,0.667
2,2019,535,U.S. Open,"Rahm, Jon",3,72,0.408,316.8,0.778,0.500
3,2019,535,U.S. Open,"Rahm, Jon",4,67,6.169,323.1,0.778,0.800
4,2019,535,U.S. Open,"Oosthuizen, Louis",1,67,6.718,301.6,0.667,0.833
...,...,...,...,...,...,...,...,...,...,...
2713,2024,535,U.S. Open,"Furr, Wilson",1,77,-3.282,304.3,0.278,0.583
2714,2024,535,U.S. Open,"Furr, Wilson",2,82,-8.213,301.9,0.500,0.200
2715,2024,535,U.S. Open,"Hervol, James",1,79,-5.282,305.8,0.222,0.529
2716,2024,535,U.S. Open,"Hervol, James",2,81,-7.213,306.4,0.500,0.182


In [99]:
event_list_url = "https://feeds.datagolf.com/historical-raw-data/event-list"
data_base_url = "https://feeds.datagolf.com/historical-raw-data/rounds"

# Define the years of interest
years_of_interest = range(2019, datetime.now().year + 1)

# The names of the major tournaments
majors_names = ['The Masters', 'U.S. Open', 'The Open Championship', 'PGA Championship']

# Fetch the event IDs for each major tournament
majors_event_ids = {year: {major: None for major in majors_names} for year in years_of_interest}

for year in years_of_interest:
    # Make the request for each year
    response = requests.get(f"{event_list_url}?file_format=json&key={api_key}")
    if response.status_code == 200:
        events = response.json()
        # Find the event IDs for the majors
        for event in events:
            if event['event_name'] in majors_names:
                majors_event_ids[year][event['event_name']] = event['event_id']
    else:
        print(f"Failed to fetch events for {year}: {response.status_code}")

# Now that we have the event IDs, we can fetch the round data for each major
all_majors_data = []

for year, events in majors_event_ids.items():
    for major, event_id in events.items():
        if event_id:  # If an event ID was found
            response = requests.get(f"{data_base_url}?tour=pga&event_id={event_id}&year={year}&file_format=json&key={api_key}")
            if response.status_code == 200:
                data = response.json()
                # Add additional info
                for record in data['scores']:
                    record['event_name'] = major
                    record['year'] = year
                all_majors_data.extend(data['scores'])
            else:
                print(f"Failed to fetch data for {major} ({year}): {response.status_code}")

# Convert all the collected data into a DataFrame
df_majors = pd.DataFrame(all_majors_data)

# Save the DataFrame to a CSV file
df_majors.to_csv('all_majors_data.csv', index=False)

# Show the DataFrame
print(df_majors.head())

Failed to fetch data for The Open Championship (2020): 400
Failed to fetch data for The Masters (2021): 400
Failed to fetch data for U.S. Open (2021): 400
Failed to fetch data for The Masters (2024): 400
Failed to fetch data for U.S. Open (2024): 400
Failed to fetch data for The Open Championship (2024): 400
Failed to fetch data for PGA Championship (2024): 400
   dg_id fin_text          player_name  \
0   5321        1         Woods, Tiger   
1  12422       T2      Johnson, Dustin   
2  16243       T2       Koepka, Brooks   
3  19895       T2   Schauffele, Xander   
4   7655       T5  Molinari, Francesco   

                                             round_1  \
0  {'course_name': 'Augusta National GC', 'course...   
1  {'course_name': 'Augusta National GC', 'course...   
2  {'course_name': 'Augusta National GC', 'course...   
3  {'course_name': 'Augusta National GC', 'course...   
4  {'course_name': 'Augusta National GC', 'course...   

                                             r

In [100]:
event_list_url = "https://feeds.datagolf.com/historical-raw-data/event-list"
data_base_url = "https://feeds.datagolf.com/historical-raw-data/rounds"

# Define the years of interest
years_of_interest = range(2019, datetime.now().year + 1)

# The names of the major tournaments
majors_names = ['The Masters', 'U.S. Open', 'The Open Championship', 'PGA Championship']

# Fetch the event IDs for each major tournament
majors_event_ids = {year: {major: None for major in majors_names} for year in years_of_interest}

for year in years_of_interest:
    for event_name, event_id in majors_event_ids[year].items():  # Use the event IDs for each major
        # Get the stats for the event
        stats = get_golf_stats('pga', event_id, year, api_key)
        # ... (rest of your stats collection and CSV writing code)
        
filename = 'full_golf_data.csv'

# Open the file and write the CSV data
with open(filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Year', 'Event ID', 'Event Name', 'Player Name', 'Round', 'Score', 'SG Total', 'Driving Distance', 'GIR', 'Scrambling'])
    
    for year in years_of_interest:
        for event_name, event_id in event_ids.items():
            # Get the stats for event
            stats = get_golf_stats('pga', event_id, year, api_key)
            
            # Check if stats is not None
            if stats:
                # Process and write data to CSV
                for player in stats['scores']:
                    for round_number, round_data in player.items():
                        if round_number.startswith('round_'):
                            writer.writerow([
                                year,
                                event_id,
                                event_name,
                                player['player_name'],
                                round_number[-1],
                                round_data['score'],
                                round_data['sg_total'],
                                round_data['driving_dist'],
                                round_data['gir'],
                                round_data['scrambling']
                            ])

print("Data collection complete. CSV file created.")

Data collection complete. CSV file created.


In [104]:
pd.read_csv('full_golf_data.csv')

,Year,Event ID,Event Name,Player Name,Round,Score,SG Total,Driving Distance,GIR,Scrambling
0,2019,NaN,The Masters,"Rahm, Jon",1,69,4.718,311.5,0.556,0.778
1,2019,NaN,The Masters,"Rahm, Jon",2,70,3.787,315.7,0.611,0.667
2,2019,NaN,The Masters,"Rahm, Jon",3,72,0.408,316.8,0.778,0.500
3,2019,NaN,The Masters,"Rahm, Jon",4,67,6.169,323.1,0.778,0.800
4,2019,NaN,The Masters,"Oosthuizen, Louis",1,67,6.718,301.6,0.667,0.833
...,...,...,...,...,...,...,...,...,...,...
10867,2024,NaN,PGA Championship,"Furr, Wilson",1,77,-3.282,304.3,0.278,0.583
10868,2024,NaN,PGA Championship,"Furr, Wilson",2,82,-8.213,301.9,0.500,0.200
10869,2024,NaN,PGA Championship,"Hervol, James",1,79,-5.282,305.8,0.222,0.529
10870,2024,NaN,PGA Championship,"Hervol, James",2,81,-7.213,306.4,0.500,0.182


Data collection complete. CSV file created.


In [119]:
years_of_interest = range(2019, datetime.now().year + 1)

event_ids_for_majors = {
    'The Masters': '14',
    'U.S. Open': '26',
    'The Open Championship': '100',
    'PGA Championship': '33'
}

# Create a dictionary with years as keys and the event IDs dictionary as values
majors_event_ids = {year: event_ids_for_majors for year in years_of_interest}




new_filename = 'test_majors_data.csv'

# Open the file and write the CSV data
with open(new_filename, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Year', 'Event ID', 'Event Name', 'Player Name', 'Round', 'Score', 'SG Total', 'Driving Distance', 'Driving Accuracy', 
                     'GIR', 'Scrambling', 'Fairway Proximity', 'Rough Proximity', 'Great Shots', 'Poor Shots', 'Tee Time', 'Final Position'])
    
    for year in years_of_interest:
        for event_name, event_id in majors_event_ids[year].items():  # Use the event IDs for each major
            # Get the stats for the event
            stats = get_golf_stats('pga', event_id, year, api_key)
            
            # Check if stats is not None
            if stats:
                # Process and write data to CSV
                for player in stats['scores']:
                    for round_number, round_data in player.items():
                        if round_number.startswith('round_'):
                            writer.writerow([
                                year,
                                event_id,
                                event_name,
                                player['player_name'],
                                round_number[-1],  # Extracts the round number
                                round_data['score'],
                                round_data['sg_total'],
                                round_data['driving_dist'],
                                round_data['driving_acc'],
                                round_data['gir'],
                                round_data['scrambling'],
                                round_data['prox_fw'],
                                round_data['prox_rgh'],
                                round_data['great_shots'],
                                round_data['poor_shots'],
                                round_data['teetime'],
                                player['fin_text']
                                
                            ])

print("Data collection complete. CSV file created.")

Data collection complete. CSV file created.


In [120]:
final_df = pd.read_csv('test_majors_data.csv')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10872 entries, 0 to 10871
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Year               10872 non-null  int64  
 1   Event ID           10872 non-null  int64  
 2   Event Name         10872 non-null  object 
 3   Player Name        10872 non-null  object 
 4   Round              10872 non-null  int64  
 5   Score              10872 non-null  int64  
 6   SG Total           10872 non-null  float64
 7   Driving Distance   10872 non-null  float64
 8   Driving Accuracy   10872 non-null  float64
 9   GIR                10872 non-null  float64
 10  Scrambling         10872 non-null  float64
 11  Fairway Proximity  10872 non-null  float64
 12  Rough Proximity    10872 non-null  float64
 13  Great Shots        10872 non-null  int64  
 14  Poor Shots         10872 non-null  int64  
 15  Tee Time           10872 non-null  object 
 16  Final Position     108

In [121]:
final_df

,Year,Event ID,Event Name,Player Name,Round,Score,SG Total,Driving Distance,Driving Accuracy,GIR,Scrambling,Fairway Proximity,Rough Proximity,Great Shots,Poor Shots,Tee Time,Final Position
0,2019,14,The Masters,"Rahm, Jon",1,69,4.718,311.5,0.714,0.556,0.778,32.546,40.025,4,2,3:06pm,1
1,2019,14,The Masters,"Rahm, Jon",2,70,3.787,315.7,0.357,0.611,0.667,37.182,34.145,9,3,7:51am,1
2,2019,14,The Masters,"Rahm, Jon",3,72,0.408,316.8,0.429,0.778,0.500,31.819,26.096,2,5,1:13pm,1
3,2019,14,The Masters,"Rahm, Jon",4,67,6.169,323.1,0.571,0.778,0.800,22.699,41.028,6,1,12:22pm,1
4,2019,14,The Masters,"Oosthuizen, Louis",1,67,6.718,301.6,0.643,0.667,0.833,34.716,43.317,5,2,3:17pm,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10867,2024,33,PGA Championship,"Furr, Wilson",1,77,-3.282,304.3,0.429,0.278,0.583,39.644,81.415,3,5,10:27am,CUT
10868,2024,33,PGA Championship,"Furr, Wilson",2,82,-8.213,301.9,0.286,0.500,0.200,42.538,45.566,1,8,2:42pm,CUT
10869,2024,33,PGA Championship,"Hervol, James",1,79,-5.282,305.8,0.357,0.222,0.529,42.107,63.878,2,7,10:27am,CUT
10870,2024,33,PGA Championship,"Hervol, James",2,81,-7.213,306.4,0.571,0.500,0.182,50.095,61.614,0,7,2:42pm,CUT


In [122]:
final_df.describe()

,Year,Event ID,Round,Score,SG Total,Driving Distance,Driving Accuracy,GIR,Scrambling,Fairway Proximity,Rough Proximity,Great Shots,Poor Shots
count,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000,10872.000000
mean,2021.500000,43.250000,2.125828,73.450331,-0.000088,302.218322,0.486594,0.565302,0.556989,33.230258,57.025892,3.554084,4.536424
std,1.707904,33.463271,1.054659,3.080426,3.040229,10.591484,0.155893,0.126036,0.182200,6.921475,19.628685,1.820937,2.132912
min,2019.000000,14.000000,1.000000,67.000000,-8.282000,271.600000,0.071000,0.222000,0.000000,18.780000,6.583000,0.000000,0.000000
25%,2020.000000,23.000000,1.000000,71.000000,-2.213000,295.200000,0.357000,0.500000,0.429000,28.174000,43.267000,2.000000,3.000000
50%,2021.500000,29.500000,2.000000,73.000000,0.169000,301.800000,0.500000,0.556000,0.556000,32.599000,54.126000,3.000000,4.000000
75%,2023.000000,49.750000,3.000000,76.000000,2.169000,308.100000,0.571000,0.667000,0.667000,37.407000,68.433000,5.000000,6.000000
max,2024.000000,100.000000,4.000000,82.000000,6.787000,347.600000,0.929000,0.889000,1.000000,56.250000,129.960000,10.000000,13.000000
